# Define the network

In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channnel , 6output channels, 5*5 conv kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation : y = Wx + b
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling  over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # if the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimentions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


##### the backward function( where gradients are computed) 
##### is automatically defined for you using autograd
##### you can use any of the tensor operations in the Tensor
##### operations in the forward function

In [2]:
# the learnable parameters of a model are returned by net.parameters()
params = list(net.parameters())
print(len(params))
for i in range(len(params)):
    print(params[i].size())
    
print("conv1's weight", params[0].size()) # conv1's weight

10
torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 400])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])
conv1's weight torch.Size([6, 1, 5, 5])


In [3]:
# lets try a random 32 * 32 imput
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.1392,  0.0086,  0.0893, -0.1539, -0.0608, -0.0318,  0.0733, -0.0570,
          0.0725,  0.0534]], grad_fn=<ThAddmmBackward>)


In [4]:
# zero the gradient buffers of all parameters and backprops with random gradients
net.zero_grad()
out.backward(torch.randn(1, 10))

# Loss Function
- A loss function takes the (output, target) pair of inputs, and computes a value that estimates how far away the ouput is from the target

In [5]:
output = net(input)
target = torch.randn(10) # a dummy target, for example
target = target.view(1, -1) # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.6526, grad_fn=<MseLossBackward>)


input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss

In [6]:
# for illustration, let us follow a few steps backward
print(loss.grad_fn) # MSEloss
print(loss.grad_fn.next_functions[0][0]) # linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # relu

# Backprop

In [7]:
# net.zero_grad() # zeros the gradient buffers of all parameters
# print('conv1.bias.grad before backword')
# print(net.conv1.bias.grad)

# loss.backward()

# print('conv1.bias.grad after backward')
# print(net.conv1.bias.grad)

- for more details 
    - https://pytorch.org/docs/stable/nn.html

# update the wieght
- the simplest update rule used in practice in the Stochastic Gradient Descent
    - weight = weight - learning_rate * gradient

In [8]:
# learning_rate = 0.01
# for f in net.parameters():
#     f.data.sub_(f.grad.data * learning_rate)

In [9]:
# torch.optim implements various update rules
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad() # zerothe gradient buffers
ouput = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() # Does the update